In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession, function_tool
import os
from pypdf import PdfReader
import gradio as gr
import json
import requests
from typing import Dict

load_dotenv(override=True)

In [ ]:
session = SQLiteSession("user_123", "conversation.db")

In [ ]:
kb=""
reader= PdfReader("knowledgebase/Musketeerly_AI_Knowledge_Base.pdf")
for page in reader.pages:
    kb +=page.extract_text()
    

In [60]:
company_name= "Musketeerly AI"
intent_classifier_instructions = f"""
You are a classification agent for Musketeerly AI. Your task is to analyze customer messages and determine:

    The user’s intent.
    Whether escalation to a human agent is required or to hand off to the support agent. You hand off to the support agent when the intent of the message is within the capabilities of the support agent, otherwise handoff to the escalation agent

Follow these instructions:

    Do not resolve issues yourself. Your sole responsibility is to route messages accurately.
    If asked about unrelated topics to the knowledge base, inform the customer that we do not offer such services and list the services we provide from the knowledge base.


Possible intents include:

    “get_customer_info”
    “ask_question”
    “request_refund”
    “cancel_order”
    “legal_complaint”
    “unknown”
You will judge whether to rout to the support agent or the escalation agent
Escalate when:

    The user appears angry, frustrated, or the issue is urgent.
    Refund, cancellation, or legal issues are mentioned.
    Intent is unclear.

Do not ask for clarification or perform tool actions. Your sole role is routing requests accurately.

Knowledge Base: {kb}"""
support_instructions = f"""
You are a customer support agent for Musketeerly AI.
you are a support agent that retrieves customer information from the database and display it to them using the email. you require the email to use the tool which retrieves the information. If the email provided by the customer does not exist in the database, inform the customer and inquire if their sure of the information provided or they can consider creating a new account. Do not suggest a new conversation after this"

Tools available:

    get_customer_info(email)

Redirect account creation to the signup page by giving the steps in accordance to the knowledge base.

In case you cannot resolve an issue (e.g., refund, cancellation, legal request, angry customer), handoff to the escalation agent


Knowledge Base:

{kb}

"""
escalation_instructions = f"""
You are the escalation agent for Musketeerly AI. Handle sensitive cases by involving a human support agent.
If the issue is a cancellation or refund, You are to ask for a short summary of the issue and the customers id once and never again ,then send a push notification containing the a crafted message based on the issue and the customers id immediately without further interaction.
But if the issue is anger related you are to drop a word to placate t he customer while sending the support email, end the conversation after that
After sending the push notification, Reassure the customer:
> “I’ve escalated this to a human support agent who will get back to you shortly.”
and end the conversation by thanking the customer for their patience. Do not engage further or attempt to resolve the issue yourself.
"""


In [56]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

airtable_token= os.getenv("AIRTABLE_TOKEN")
table_name= os.getenv("TABLE_NAME")
base_id= os.getenv("BASE_ID")

AIRTABLE_URL = f"https://api.airtable.com/v0/{base_id}/{table_name}"
HEADERS = {
    "Authorization": f"Bearer {airtable_token}",
    "Content-Type": "application/json"
}


In [57]:
@function_tool
def escalate_to_human(issue_summary:str):
    """Sends a Push Notification For escalation of issues. Must require a customer id to be included in the message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": issue_summary}
    requests.post(pushover_url, data=payload)



# Tool 1: Get Customer by Email
# ----------------------------
@function_tool
def get_customer_info(email: str):
    
    params = {
        "filterByFormula": f"{{Email}}='{email}'",
        "maxRecords": 1
    }
    response = requests.get(AIRTABLE_URL, headers=HEADERS, params=params)
    response.raise_for_status()
    data = response.json()
    
    if data.get("records"):
        return data["records"][0]  # Return first matching record
    else:
        return None




In [58]:
escalation_agent = Agent(name="Escalation Agent", instructions = escalation_instructions, model= "gpt-4o-mini", tools = [escalate_to_human], handoff_description=f"""You are the Escalation Agent for {company_name}. You handle situations that require human involvement or sensitive support cases such as refunds, cancellations, legal complaints, or very frustrated customers.

You will receive escalated cases from the Support Agent or the intent agent. Your job is to escalate the issue to the human agent

""")

support_agent= Agent(name="Support Agent", instructions= support_instructions, model= "gpt-4o-mini", tools=[get_customer_info],handoffs=[escalation_agent], handoff_description=f"""You are the Support Agent for customer service at {company_name}. Your responsibility is to resolve routine customer issues using available tools.

You can independently handle the following types of requests:
- Answering frequently asked questions using the knowledge base
- Updating customer account information such as address, phone, or email
- retrieving customer account information

Tools available to you:
- update_customer_info(email, field, value). email must be needed
- get_customer_info(email). email must is needed

""")


intent_agent= Agent(name="Intent Classifier Agent", instructions= intent_classifier_instructions, model= "gpt-4o-mini", handoffs=[support_agent, escalation_agent])





In [61]:
async def chat(message, history):
    with trace("Musketeerly"):
        response = await Runner.run(intent_agent, message, session=session)
        return response.final_output
gr.ChatInterface(chat).launch()







c:\Users\USER\Musketeerly\.venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
